<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021_0301ccap_word2vec_onomatopea_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語オノマトペ表現の単語ベクトル (word2vec) モデルによる理解

- date: 2021-0301
- author: 浅川伸一
- lincense: MIT license
- filename 2021kondo_onomatopea.ipynb

使い方の詳細は内部にコメントとして記述しています。
定義した射影空間，と <font color="teal" size="+1">歪んだ</font>射影空間への射影ベクトルを使った最近接語を表示します。



In [ ]:
#ひとつ下の '日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
#別の方法で配布させていただきます。
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
import sys
from scipy import stats
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

!pip install jaconv
import jaconv  # ひらがなカタカナ変換用 `pip install jaconv` してください

#import matplotlib.pyplot as plt
#import japanize_matplotlib  # matplotlib の日本語表示
#%matplotlib inline


# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータ
# 著作権の問題が不明のため colab で利用可能データにしていませんのでご注意ください
# データ置き場
#ccap_base = '/Users/asakawa/study/2021ccap/notebooks'
ccap_base = '.'

# xlsx ファイルでは読み込めないため xls ファイルに変換して利用
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onomatopea_excel = '日本語オノマトペ辞典4500より.xls'

# 実際のエクセルファイルの読み込み
onmtp2761 = pd.read_excel(os.path.join(ccap_base, onomatopea_excel), sheet_name='2761語')

# 読み込んだデータ確認のため印字
print(len(list(onmtp2761['オノマトペ'])), ': データ数')
first_n = 5  # 最初の first_n 個のデータをテスト表示
print(list(onmtp2761['オノマトペ'])[:first_n], ':最初の', first_n, 'データを表示')

In [ ]:
# word2vec データの読み込み
# ローカルディスクから読み込むようになっています。colab でお使いの場合には
# 適宜変更してください
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz'
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_base = '/Users/asakawa/study/2016wikipedia/'
w2v_base = '.'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'

In [ ]:
# word2vec の読み込み準備
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [ ]:
# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります
# 2つ上のセルと同様，ローカルドライブから読み込むようになっています。
#w2v_base = '/Users/asakawa/study/2016wikipedia/'
w2v_base = '.'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 
w2v = asakawa_w2v

In [ ]:
# オノマトペのうち，word2vec に登録があるかどうかを調査
kana_entries = []
kata_entries = []
count = 0
for word in list(onmtp2761['オノマトペ']):
    count += 1
    if word in w2v.vocab:
        kana_entries.append(word)
    else:
        # オノマトペが word2vec 項目にない場合はカタカナに変換する
        kata_w = jaconv.hira2kata(word)
        if kata_w in w2v.vocab:
            kata_entries.append(kata_w)
            #onmtp2761['オノマトペ'][word] = kata_w

print('There are', len(set(kana_entries)), 'kana onomatopea words in jawikipedia word2vec')
print('There are', len(set(kata_entries)), 'katakana onomatopea words in jawkipedia word2vec.')

# entries 
entries = list(set(kana_entries + kata_entries))
print('There are {} entries in total.'.format(len(entries)))

---

ここから下がシミュレーションの本体です。
---

In [ ]:
class w2v_subspace():
    """射影行列の表示
    少し内容を変更しました。クラス初期化時に引数 entries に任意の単語リストを入れると
    その単語リストで構成される部分空間を構成するように変更しました。
    
    また，結果呼び出し時に 引数 verbose=False にすると各エントリーについて，それぞれの
    射影行列 4 種とオリジナル word2vec による最近接語を topn にしてされた個数だけ入れた
    リストから構成される dict を返すようにしました
    
    
    # w2v_subspace():
    w2v: class w2v
        gensim 形式の訓練済 word2vec クラス
    entries: list
        部分空間を構成する単語のリスト
        word2vec に該当する単語がなければ無視する
        
    # _call_(targets, verbose):
    tarets: [str,]
        調べたい単語リスト
        例: targets=['トラ', 'イヌ', 'ペロペロ']
    topn: (int)
        最近接語数
    verbose: (bool)
        True: その都度結果を印字する
        Flase: 結果は dict 型データに格納して返す
    """
    
    def __init__(self, w2v=w2v, entries=None):
        """オノマトペリスト wordlist はエクセルファイルであることを仮定
        """
        self.w2v = w2v
        self.count = 0
        if entries != None:
            self.entries = wordlist
            self.count = len(self.entries)
        else:
            # オノマトペのうち，word2vec に登録があるかどうかを調査
            kana_entries, kata_entries = [], []
            for word in list(onmtp2761['オノマトペ']):
                self.count += 1
                
                if word in w2v.vocab:
                    # word2vec にエントリがある場合
                    kana_entries.append(word)
                else:
                    # オノマトペが word2vec 項目にない場合はカタカナに変換
                    kata_w = jaconv.hira2kata(word)
                    if kata_w in w2v.vocab:
                        # カタカナのエントリ存在する場合
                        kata_entries.append(kata_w)
        
            self.entries = list(set(kana_entries + kata_entries))

        # 射影行列 P を初期化    
        self.P = np.zeros((len(set(self.entries)), w2v.vector_size))
        
        # 射影空間の作成
        for i, x in enumerate(set(self.entries)):
            self.P[i] = np.copy(self.w2v[x])
            
        # 内積逆行列 (X^{T} X)^{-1} の作成
        invP = np.linalg.inv(np.dot(self.P, self.P.T))
        
        # 直交射影行列 Proj の作成
        self.Proj = np.dot(self.P.T, np.dot(invP, self.P))
        
        # 歪んだ射影行列の作成 2021年2月23日 近藤公久先生とのディスカッション
        # で解釈可能なら残しても良いということで使ってみる
        self.wProj = np.dot(self.P.T, self.P)

        # 直交補空間への射影行列の作成
        I = np.eye(len(self.Proj))
        self.C = I - self.Proj
        self.wC = I - self.wProj
        return

    def __call__(self, targets=['電車','ネコ','東京'], topn=5, verbose=True):
        ret = {}
        for target in targets:
            if not target in self.w2v:
                # もし word2vec にエントリが無かったらカタカナに変換
                k_target = jaconv.hira2kata(target)
                if k_target in self.w2v:
                    target = k_target
                else:
                    continue
                    
            # target 語の意味ベクトルを取得
            x = np.array(self.w2v[target])
            
            # 射影ベクトルを計算
            wPx = np.dot(self.wProj, x)
            wPx_list = [w[0] for w in list(self.w2v.similar_by_vector(wPx,topn=topn))]
            
            wPxc = np.dot(self.wC, x)
            wPxc_list = [w[0] for w in list(self.w2v.similar_by_vector(wPxc,topn=topn))]

            Px = np.dot(self.Proj, x)
            Px_list = [w[0] for w in self.w2v.similar_by_vector(Px,topn=topn)]

            Cx = np.dot(self.C, x)
            Cx_list = [w[0] for w in w2v.similar_by_vector(Cx,topn=topn)]
            
            w2v_list = [w[0] for w in list(self.w2v.similar_by_vector(x,topn=topn))]
            
            ret[target] = {
                'word2vec': w2v_list,
                'wProj': wPx_list,
                'wProj_c': wPxc_list,
                'Proj': Px_list,
                'Proj_c': Cx_list
            }

            if verbose:
                print('word2vec 最近隣語({0}):{1}'.format(target, w2v_list))
                print('         w付き射影ベクトル({0}):{1}'.format(target, wPx_list))
                print('        w付き補射影ベクトル({0}):{1}'.format(target, wPxc_list))
                print('          直交射影ベクトル({0}):{1}'.format(target, Px_list))
                print('         直交射影補ベクトル({0}):{1}'.format(target, Cx_list))
        
        if verbose:
            return
        else:
            return ret
        
P = w2v_subspace()
P(topn=7)


In [ ]:
P(targets=['がたがた', 'ガタガタ', 'どきどき','ドキドキ', 'つるつる', 'ツルツル','すべすべ', 'スベスベ'])

In [ ]:
P(targets=['電車','ネコ','男性'])

In [ ]:
# 確認用，word2vec に登録されている単語を任意に n_test だけ調べて表示する
n_test = 5
for i in range(5):
    word = np.random.choice(list(w2v.vocab.keys()))
    P([word])


In [ ]:
kawabe_tab1 = ['きーん', 'じわじわ', 'じんじん', 'すーすー', 'ぽかぽか']
P(kawabe_tab1)

In [ ]:
kawabe_tab2 = (# 自然-天気
    'かんかん', 'はらはら', 'ごろごろ', 'ばらばら', 'ざーざー', 'ぱらぱら', 'さーっ',
    'びしゃびしゃ', 'ざーっ', 'ひゅー', 'さーさー', 'ひゅーひゅー', 'じりじり', 'びゅーびゅー',
    'そよそよ', 'びゅんびゅん',  'ばたばた', 'ぴゅんぴゅん')
P(kawabe_tab2)

In [ ]:
kawabe_tab3 = (# 自然 火・土
    'がたがた', 'ぱりぱり', 'がんがん', 'べちょっ', 'ごつごつ', 'ぼーっ', 'さくさく',
    'ぼーぼー', 'さくっ', 'ぼこぼこ', 'じゃりじゃり', 'ぼっ', 'ぱちぱち')
P(kawabe_tab3)

In [ ]:
kawabe_tab4 = (# 自然 水・液体
    'がばがば', 'ぱしゃっ', 'ころころ', 'ばちゃっ', 'ざばっ', 'びしゃっ', 'さらさら',
    'ぴしゃっ', 'ざぶん', 'びちゃっ', 'じゃーじゃー', 'びちゃびちゃ', 'じゃぶん',
    'びちょびちょ', 'じゃぼん', 'ぶくぶく', 'だぼだぼ', 'ぷくぷく', 'たぷんたぷん', 
    'べちゃべちゃ', 'ちゃぷちゃぷ', 'ぼたぼた', 'ちゅっ', 'ぽたぽた', 'ちょろちょろ', 
    'ぽつん', 'どくどく', 'ぼとぼと', 'どぼどぼ', 'ぽとん', 'どぼん', 'ぽとぽと')

kawabe_tab7 = {# 両方の意味をもつ高認知度の単語 20 語 自然分類ごと
    '温度': ('ぽかぽか',),
    '天気': ('ごろごろ', 'ざーっ', 'じりじり', 'ぱらぱら', 'ひゅー', 'ひゅーひゅー', 'びゅんびゅん'),
    '火・土':('ごつごつ', 'じゃりじゃり', 'ぱちぱち', 'べちょっ', 'ぼーぼー', 'ぼこぼこ'),
    '水・液体':('さらさら', 'ちょろちょろ', 'ぱしゃっ', 'びちょびちょ', 'べちゃべちゃ', 'ぽつん')}

kawabe_tab8 = (#表 8 実験に使用したオノマトペ 10 語
    'がたがた', 'じりじり', 'かんかん', 'ばたばた', 'ごろごろ', 'ぱちぱち', 'ざーっ', 'ぶくぶく', 
    'さくさく', 'ぼーっ')

kawabe_tab9 = (# 表 9 実験に使用した多義語 10 語
    'さっぱり', '青い', 'そろそろ', 'メジャー', '鋭い', '確か',
    '相当', '甘い', '一体', '勝手')

print(P(kawabe_tab4))
print(P(kawabe_tab7))
print(P(kawabe_tab8))
print(P(kawabe_tab9))

In [ ]:
kawabe_tab1 = (#自然-温度
'きーん', 'じわじわ', 'じんじん', 'すーすー', 'ぽかぽか')
P(kawabe_tab1)

In [ ]:
words = ['うさぎ', 'あじさい', 'ワニ', '天ぷら', 'フクロウ', 'みかん', 
         'かご', 'バラ', 'へび', 'いちご', 'ソバ', 'おなか',
         'オレンジ', 'ハト', 'のど仏', '刺身', 'にわとり', 'スリコギ',
         'ぶどう', 'イチョウ', 'びわ', '手すり', '風車', '鋏',
         'ごはん', 'クジラ', 'タイ焼き', '靴べら', 'タンポポ', 'ヤギ',
         'エビ', 'かぶ', 'まんじゅう', 'リンゴ', 'タツノオトシゴ', 'レンコン']
P(words)

In [ ]:
kawabe_taba1=(#表 A1 予備実験に使用したオノマトペ','意味分類','自然-温度(9 語)
#自然-温度
'きーん', 'ぽかぽか', 'ぎんぎん', 'ぽかりぽかり', 'じわじわ', 'ぽっぽっ', 'じんじん', 'りん', 'すーすー')

print(P(kawabe_taba1))

In [ ]:
kawaba_taba2 = (# 表 A2 予備実験に使用したオノマトペ','意味分類','自然-天気(45 語)
#自然-天気
'かっ','ざんざら','びしゃびしゃ',
'かっか','ざんざ','ひゅー',
'からっ','しゃりしゃり','びゅー',
'からり','じりじり','ぴゅー',
'かんかん','すーっ','ひゅーひゅー',
'ごろごろ','そよそよ','びゅーびゅー',
'こんこん','そよ','ぴゅーぴゅー',
'ざーざー','そより','ひゅっ',
'さーっ','どしゃどしゃ','ぴゅっ',
'ざーっ','はたはた','びゅんびゅん',
'さーさー','ばたばた','ぴゅんぴゅん',
'ざざっ','ぱたぱた','ふー',
'ざっ','はらはら','ぶぉーっ',
'さやさや','ばらばら','ぶんぶん',
'さわさわ','ぱらぱら','みりり'
)

print(P(kawabe_tab2))

In [ ]:
kawabe_taba3 = (#表 A3 予備実験に使用したオノマトペ','意味分類','自然-火・土(22 語)
#自然-火・土
'がたがた','ぱちぱち',
'がんがん','ぱりぱり',
'ごつごつ','ぶすぶす',
'さくさく','ぷすぷす',
'さくっ','べちょっ',
'さくりさくり','ぼーっ',
'ざっく','ぼーぼー',
'じゃくり','ぼーん',
'じゃりじゃり','ぼこぼこ',
'ずぶずぶ','ぼっ',
'ちろちろ','もー')

print(P(kawabe_taba3))


In [ ]:
kawabe_taba4 = (#表 A4 予備実験に使用したオノマトペ	意味分類	自然-水・液体(55 語)
#自然-水・液体
'がばがば','ずっぷり','とぷんとぷん','しょろしょろ',
'がぼがぼ','だぼだぼ','どぼん','ぷくぷく',
'ころころ','たぷんたぷん','ばしゃっ','べちゃべちゃ',
'ざばっ','たぽたぽ','ぱしゃっ','ぼしゃっ',
'ざぶっ','ちゃぷちゃぷ','ばちゃっ','ぼたぼた',
'さらさら','ちゅっ','ぱちゃっ','ぽたり',
'ざぶん','ちょろちょろ','びしゃっ','ぽたん',
'ざんぶり','どーっ','ぴしゃっ','ぽたぽた',
'じくじく','どーどー','ひたひた','ぽつん',
'しゃーしゃー','とくとく','びちゃっ','ぼとぼと',
'じゃーじゃー','どくどく','ぴちゃっ','ぽとり',
'じゃぶん','どぶどぶ','びちゃびちゃ','ぽとん',
'じゃぼん','どぼどぼ','びちょびちょ','ぽとぽと',
'じゅくじゅく','どぶん','ぶくぶく')

print(P(kawabe_taba4))